In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install fvcore tqdm matplotlib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/xpuria/MLDL2024_project.git

fatal: destination path 'MLDL2024_project' already exists and is not an empty directory.


In [3]:
# Create project directories
!mkdir -p datasets/cityscapes datasets/gta5 checkpoints plots

# First, let's extract the datasets
import os
import zipfile
from pathlib import Path

# Extract Cityscapes
print("Extracting Cityscapes...")
cityscapes_zip = '/content/drive/MyDrive/Cityscapes.zip'
cityscapes_dir = '/content/datasets/cityscapes'

with zipfile.ZipFile(cityscapes_zip, 'r') as zip_ref:
    zip_ref.extractall(cityscapes_dir)

# Extract GTA5
print("Extracting GTA5...")
gta5_zip = '/content/drive/MyDrive/GTA5.zip'
gta5_dir = '/content/datasets/gta5'

with zipfile.ZipFile(gta5_zip, 'r') as zip_ref:
    zip_ref.extractall(gta5_dir)

# Verify the structure
def print_directory_structure(path, level=0):
    indent = '  ' * level
    print(f"{indent}{os.path.basename(path)}/")
    if level < 2:  # Limit depth to avoid too much output
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print_directory_structure(item_path, level + 1)

print("\nDataset structure:")
print_directory_structure('/content/datasets')

# Print dataset statistics
print("\nDataset statistics:")
print("Cityscapes:")
print(f"Total files: {len(list(Path(cityscapes_dir).rglob('*.*')))}")
print("\nGTA5:")
print(f"Total files: {len(list(Path(gta5_dir).rglob('*.*')))}")

Extracting Cityscapes...
Extracting GTA5...

Dataset structure:
datasets/
  cityscapes/
    Cityscapes/
  gta5/
    GTA5/

Dataset statistics:
Cityscapes:
Total files: 6216

GTA5:
Total files: 5000


In [4]:
# First, uninstall existing sympy and reinstall the correct version
!pip uninstall -y sympy
!pip install sympy==1.12

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 106.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [11]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_project.datasets.cityscapes import CityscapesDataset
from MLDL2024_project.datasets.gta5 import GTA5Dataset
from MLDL2024_project.utils import test_fps_latency, count_parameters, analyze_model_complexity

In [ ]:
# Test Cityscapes dataset
def test_cityscapes():
    dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='train',
        img_size=(512, 1024)
    )

    loader = DataLoader(dataset, batch_size=2, shuffle=True)
    images, labels = next(iter(loader))

    print("\nCityscapes test:")
    print(f"Dataset size: {len(dataset)}")
    print(f"Image shape: {images.shape}")
    print(f"Label shape: {labels.shape}")
    print(f"Image value range: ({images.min():.2f}, {images.max():.2f})")
    print(f"Label unique values: {torch.unique(labels).numpy()}")

    return images, labels

# Test GTA5 dataset
def test_gta5():
    dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280
    )

    loader = DataLoader(dataset, batch_size=2, shuffle=True)
    images, labels = next(iter(loader))

    print("\nGTA5 test:")
    print(f"Dataset size: {len(dataset)}")
    print(f"Image shape: {images.shape}")
    print(f"Label shape: {labels.shape}")
    print(f"Image value range: ({images.min():.2f}, {images.max():.2f})")
    print(f"Label unique values: {torch.unique(labels).numpy()}")

    return images, labels

# Run tests
cityscapes_images, cityscapes_labels = test_cityscapes()
gta5_images, gta5_labels = test_gta5()



Cityscapes train Dataset:
Number of samples: 1572
Image size: (512, 1024)
Augmentations: False
Cities (10): hanover, jena, krefeld, monchengladbach, strasbourg, stuttgart, tubingen, ulm, weimar, zurich

Cityscapes test:
Dataset size: 1572
Image shape: torch.Size([2, 3, 512, 1024])
Label shape: torch.Size([2, 512, 1024])
Image value range: (-1.93, 2.64)
Label unique values: [  0   1   2   4   5   6   7   8   9  10  11  13  16  18 255]

GTA5 test:
Dataset size: 2500
Image shape: torch.Size([2, 3, 720, 1280])
Label shape: torch.Size([2, 720, 1280, 3])
Image value range: (-2.03, 2.64)
Label unique values: [  0  20  30  35  60  64  70  74  81 100 102 107 110 111 128 130 142 150
 152 153 156 170 180 190 220 230 232 244 250 251 255]


In [3]:
from MLDL2024_project.models.deeplabv2.deeplabv2 import get_deeplab_v2

In [ ]:
cityscapes_train_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='train',
        img_size=(512, 1024)
    )
cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes train Dataset:
Number of samples: 1572
Image size: (512, 1024)
Augmentations: False
Cities (10): hanover, jena, krefeld, monchengladbach, strasbourg, stuttgart, tubingen, ulm, weimar, zurich

Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [7]:
DeepLabV2_model = get_deeplab_v2(pretrain_model_path="/content/drive/MyDrive/deep_lab_model/deeplab_resnet_pretrained_imagenet.pth").to(device)

Deeplab pretraining loading...


/content/MLDL2024_project/models/deeplabv2/deeplabv2.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(pretrain_model_path)


In [ ]:
DeepLabV2_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
DeepLabV2_optimizer = torch.optim.SGD(DeepLabV2_model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-4)

In [ ]:
class_names = cityscapes_train_dataset.get_class_names()

In [ ]:
from MLDL2024_project.train import train_model

In [ ]:
import numpy as np

In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=DeepLabV2_model,
    criterion=DeepLabV2_criterion,
    optimizer=DeepLabV2_optimizer,
    train_loader=cityscapes_train_dataloader,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:21<00:00, 11.90it/s, Loss=0.4378]



Epoch 1/50:
Train Loss: 0.6353, mIoU: 28.39%

Train IoU per class:
road                : 88.85%
sidewalk            : 48.35%
building            : 73.36%
wall                : 7.06%
fence               : 9.88%
pole                : 13.75%
traffic light       : 0.44%
traffic sign        : 15.87%
vegetation          : 73.75%
terrain             : 28.27%
sky                 : 75.06%
person              : 27.61%
rider               : 1.01%
car                 : 62.32%
truck               : 0.05%
bus                 : 0.62%
train               : 0.17%
motorcycle          : 0.21%
bicycle             : 12.88%

Val Loss: 0.4724, mIoU: 33.25%

Val IoU per class:
road                : 91.93%
sidewalk            : 48.69%
building            : 75.29%
wall                : 3.69%
fence               : 7.69%
pole                : 21.52%
traffic light       : 3.88%
traffic sign        : 25.27%
vegetation          : 78.49%
terrain             : 38.07%
sky                 : 83.16%
person              :

Epoch 2/50 (Val): 100%|██████████| 250/250 [00:20<00:00, 11.96it/s, Loss=0.3836]



Epoch 2/50:
Train Loss: 0.3158, mIoU: 42.95%

Train IoU per class:
road                : 94.32%
sidewalk            : 68.40%
building            : 83.59%
wall                : 24.21%
fence               : 26.15%
pole                : 27.37%
traffic light       : 11.51%
traffic sign        : 35.43%
vegetation          : 84.01%
terrain             : 46.85%
sky                 : 86.76%
person              : 51.44%
rider               : 15.21%
car                 : 83.20%
truck               : 9.24%
bus                 : 7.40%
train               : 7.15%
motorcycle          : 12.11%
bicycle             : 41.67%

Val Loss: 0.3480, mIoU: 41.82%

Val IoU per class:
road                : 95.00%
sidewalk            : 64.48%
building            : 80.50%
wall                : 23.75%
fence               : 21.03%
pole                : 25.32%
traffic light       : 16.24%
traffic sign        : 36.44%
vegetation          : 80.95%
terrain             : 44.46%
sky                 : 86.36%
person       

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:20<00:00, 11.94it/s, Loss=0.2403]


...


Epoch 48/50:
Train Loss: 0.0289, mIoU: 93.10%

Train IoU per class:
road                : 99.57%
sidewalk            : 97.20%
building            : 97.75%
wall                : 95.72%
fence               : 95.34%
pole                : 72.67%
traffic light       : 82.71%
traffic sign        : 89.08%
vegetation          : 97.10%
terrain             : 94.79%
sky                 : 97.17%
person              : 90.91%
rider               : 86.74%
car                 : 98.03%
truck               : 97.71%
bus                 : 96.95%
train               : 98.28%
motorcycle          : 92.06%
bicycle             : 89.09%

Val Loss: 0.3188, mIoU: 61.56%

Val IoU per class:
road                : 97.19%
sidewalk            : 77.94%
building            : 87.54%
wall                : 42.39%
fence               : 37.78%
pole                : 40.16%
traffic light       : 40.66%
traffic sign        : 57.77%
vegetation          : 88.39%
terrain             : 56.40%
sky                 : 90.88%
person   

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:21<00:00, 11.86it/s, Loss=0.4760]



Epoch 49/50:
Train Loss: 0.0286, mIoU: 93.18%

Train IoU per class:
road                : 99.57%
sidewalk            : 97.25%
building            : 97.77%
wall                : 95.79%
fence               : 95.34%
pole                : 72.90%
traffic light       : 82.89%
traffic sign        : 89.20%
vegetation          : 97.14%
terrain             : 94.89%
sky                 : 97.20%
person              : 90.99%
rider               : 86.96%
car                 : 98.05%
truck               : 97.76%
bus                 : 97.07%
train               : 98.29%
motorcycle          : 92.00%
bicycle             : 89.26%

Val Loss: 0.3173, mIoU: 61.69%

Val IoU per class:
road                : 97.18%
sidewalk            : 77.86%
building            : 87.49%
wall                : 43.74%
fence               : 37.99%
pole                : 39.65%
traffic light       : 40.31%
traffic sign        : 57.22%
vegetation          : 88.38%
terrain             : 56.15%
sky                 : 90.87%
person   

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:21<00:00, 11.87it/s, Loss=0.3301]



Epoch 50/50:
Train Loss: 0.0283, mIoU: 93.25%

Train IoU per class:
road                : 99.58%
sidewalk            : 97.28%
building            : 97.79%
wall                : 95.88%
fence               : 95.50%
pole                : 73.02%
traffic light       : 83.14%
traffic sign        : 89.25%
vegetation          : 97.16%
terrain             : 94.92%
sky                 : 97.21%
person              : 91.07%
rider               : 87.16%
car                 : 98.07%
truck               : 97.80%
bus                 : 97.10%
train               : 98.35%
motorcycle          : 92.14%
bicycle             : 89.31%

Val Loss: 0.3169, mIoU: 61.62%

Val IoU per class:
road                : 97.19%
sidewalk            : 77.86%
building            : 87.57%
wall                : 42.84%
fence               : 37.54%
pole                : 39.93%
traffic light       : 41.10%
traffic sign        : 57.71%
vegetation          : 88.45%
terrain             : 56.10%
sky                 : 90.78%
person   

In [ ]:
train_miou

[28.394608243250712,
 42.94825594471116,
 52.303669770535286,
 59.29714727589956,
 65.41216652950025,
 69.95395925863892,
 73.67054499279865,
 76.10237712017226,
 78.31612247211075,
 79.7307565098684,
 80.9748766107653,
 81.68345903842126,
 81.96440585526838,
 83.3251967496706,
 84.0344688016645,
 84.62207139917416,
 85.4759409754835,
 85.98763678876654,
 86.6618416509879,
 87.04927091714386,
 87.30032924965387,
 87.4528315083522,
 87.4303424276998,
 88.23128972158469,
 88.50797036352104,
 88.93680298070964,
 89.29794600507736,
 89.60813495009138,
 89.89054879280113,
 90.10886329700853,
 90.35878028258341,
 90.61337803747168,
 90.81867219392133,
 91.03713480760769,
 91.2056042604381,
 91.35815414365146,
 91.57198617249364,
 91.79297709143549,
 91.9438821153121,
 92.1105178413027,
 92.29668821804644,
 92.44915581013494,
 92.5573942970388,
 92.7089369521068,
 92.81944162295713,
 92.93992555552846,
 93.01127279962895,
 93.0991541510214,
 93.17544513311266,
 93.24953392310714]

In [ ]:
[i for i,j in enumerate(train_miou) if j==max(train_miou)]

[49]

In [ ]:
val_miou

[33.24873011377807,
 41.823025065901795,
 47.78084426978511,
 47.35907236256207,
 55.092007229129294,
 56.90288859942606,
 55.20452119923051,
 58.01923279348038,
 58.29720839936788,
 59.34487107643821,
 59.61343277821304,
 59.66763698896722,
 60.33357290542797,
 59.56223091638822,
 59.21602912553598,
 60.56055668199014,
 60.77291083300671,
 60.9426093346261,
 61.01321366947502,
 60.784873657063606,
 61.030685904279466,
 55.05080513609734,
 60.64103401584,
 60.40653626732389,
 61.01350857380364,
 60.83863781816389,
 61.48717501458821,
 60.9383914031808,
 61.91548898351535,
 61.41879365881725,
 61.60754164849923,
 60.63198094151132,
 61.54772758441223,
 61.01036049550848,
 61.239624983998056,
 61.341966775803215,
 62.176972581200495,
 61.710504843988275,
 61.43872074745907,
 61.68068400219548,
 61.3860352388741,
 61.50295073107791,
 61.499785909602465,
 61.61115993291028,
 61.617735996366804,
 61.43757147475114,
 61.678242562464,
 61.55553835208738,
 61.685912630649945,
 61.6218219473417

In [ ]:
best_epoch

36

In [12]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [13]:
DeepLabV2_model = get_deeplab_v2().to(device)

Deeplab pretraining loading...


/content/MLDL2024_project/models/deeplabv2/deeplabv2.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(pretrain_model_path)


In [ ]:
print(test_fps_latency(DeepLabV2_model, device))

{'mean_latency': 23.95434856414795, 'std_latency': 5.3014979634335075, 'min_latency': 23.030996322631836, 'max_latency': 76.62653923034668, 'fps': 41.74607367518574}


In [ ]:
print(count_parameters(DeepLabV2_model))

{'total_params': 43901068, 'trainable_params': 43795724, 'params_M': 43.901068}


In [15]:
from fvcore.nn import FlopCountAnalysis, flop_count_table

In [16]:
def test_FLOPs_params(model, device, height, width):
    image = torch.zeros(8, 3, height, width).to(device)
    flops = FlopCountAnalysis(model, image)
    return flop_count_table(flops)

In [18]:
print(test_FLOPs_params(DeepLabV2_model, device, 64, 128))

| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.901M                | 54.799G    |
|  conv1                         |  9.408K                |  0.154G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  5.243M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  0.988G    |
|   layer1.0                     |   75.008K              |   0.345G   |
|    layer1.0.conv1              |    4.096K              |    18.383M |
|    layer1.0.bn1                |    0.128K              |    1.436M  |
|    layer1.0.conv2              |    36.864K             |    0.165G  |
|    layer1.0.bn2                |    0.128K       

In [19]:
from MLDL2024_project.models.bisenet.build_bisenet import BiSeNet

In [20]:
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 207MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 227MB/s]


In [ ]:
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=BiSeNet_model,
    criterion=BiSeNet_criterion,
    optimizer=BiSeNet_optimizer,
    train_loader=cityscapes_train_dataloader,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.41it/s, Loss=0.4590]



Epoch 1/50:
Train Loss: 1.5719, mIoU: 20.63%

Train IoU per class:
road                : 83.68%
sidewalk            : 32.25%
building            : 64.79%
wall                : 0.19%
fence               : 0.03%
pole                : 0.00%
traffic light       : 0.00%
traffic sign        : 0.15%
vegetation          : 65.21%
terrain             : 19.64%
sky                 : 69.73%
person              : 5.73%
rider               : 0.00%
car                 : 50.59%
truck               : 0.00%
bus                 : 0.02%
train               : 0.00%
motorcycle          : 0.00%
bicycle             : 0.01%

Val Loss: 0.6414, mIoU: 24.06%

Val IoU per class:
road                : 89.12%
sidewalk            : 39.91%
building            : 71.98%
wall                : 1.52%
fence               : 0.01%
pole                : 0.20%
traffic light       : 0.00%
traffic sign        : 1.84%
vegetation          : 76.33%
terrain             : 26.30%
sky                 : 74.72%
person              : 22.49

Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.53it/s, Loss=0.4431]



Epoch 2/50:
Train Loss: 1.0283, mIoU: 27.07%

Train IoU per class:
road                : 90.35%
sidewalk            : 52.02%
building            : 74.75%
wall                : 4.84%
fence               : 2.27%
pole                : 0.61%
traffic light       : 0.00%
traffic sign        : 4.43%
vegetation          : 76.22%
terrain             : 35.54%
sky                 : 78.38%
person              : 26.14%
rider               : 0.00%
car                 : 68.30%
truck               : 0.00%
bus                 : 0.00%
train               : 0.00%
motorcycle          : 0.00%
bicycle             : 0.55%

Val Loss: 0.4896, mIoU: 29.05%

Val IoU per class:
road                : 92.62%
sidewalk            : 55.21%
building            : 76.00%
wall                : 13.40%
fence               : 4.83%
pole                : 4.32%
traffic light       : 0.00%
traffic sign        : 4.25%
vegetation          : 79.94%
terrain             : 35.61%
sky                 : 81.73%
person              : 30.

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.27it/s, Loss=0.5299]


...


Epoch 48/50:
Train Loss: 0.2114, mIoU: 80.46%

Train IoU per class:
road                : 98.56%
sidewalk            : 90.98%
building            : 93.99%
wall                : 82.63%
fence               : 81.49%
pole                : 48.49%
traffic light       : 50.26%
traffic sign        : 68.44%
vegetation          : 92.84%
terrain             : 83.30%
sky                 : 94.64%
person              : 77.23%
rider               : 59.73%
car                 : 93.94%
truck               : 89.64%
bus                 : 87.52%
train               : 92.54%
motorcycle          : 73.03%
bicycle             : 69.57%

Val Loss: 0.2797, mIoU: 53.72%

Val IoU per class:
road                : 96.40%
sidewalk            : 73.80%
building            : 85.98%
wall                : 43.42%
fence               : 29.50%
pole                : 35.30%
traffic light       : 31.75%
traffic sign        : 49.61%
vegetation          : 87.49%
terrain             : 50.48%
sky                 : 90.80%
person   

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.45it/s, Loss=0.1768]



Epoch 49/50:
Train Loss: 0.2101, mIoU: 80.52%

Train IoU per class:
road                : 98.57%
sidewalk            : 91.06%
building            : 94.01%
wall                : 82.72%
fence               : 81.83%
pole                : 48.70%
traffic light       : 50.56%
traffic sign        : 68.45%
vegetation          : 92.84%
terrain             : 83.41%
sky                 : 94.66%
person              : 77.42%
rider               : 59.77%
car                 : 93.95%
truck               : 89.96%
bus                 : 87.31%
train               : 92.71%
motorcycle          : 72.45%
bicycle             : 69.55%

Val Loss: 0.2823, mIoU: 53.36%

Val IoU per class:
road                : 96.30%
sidewalk            : 73.54%
building            : 86.10%
wall                : 40.49%
fence               : 30.05%
pole                : 35.48%
traffic light       : 30.92%
traffic sign        : 49.53%
vegetation          : 87.45%
terrain             : 50.43%
sky                 : 90.79%
person   

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.56it/s, Loss=0.4083]



Epoch 50/50:
Train Loss: 0.2095, mIoU: 80.59%

Train IoU per class:
road                : 98.58%
sidewalk            : 91.10%
building            : 94.03%
wall                : 83.03%
fence               : 81.94%
pole                : 48.73%
traffic light       : 49.92%
traffic sign        : 68.47%
vegetation          : 92.87%
terrain             : 83.50%
sky                 : 94.67%
person              : 77.38%
rider               : 59.88%
car                 : 93.97%
truck               : 90.14%
bus                 : 87.50%
train               : 92.47%
motorcycle          : 73.40%
bicycle             : 69.69%

Val Loss: 0.2790, mIoU: 53.47%

Val IoU per class:
road                : 96.34%
sidewalk            : 73.69%
building            : 86.07%
wall                : 42.32%
fence               : 30.48%
pole                : 35.31%
traffic light       : 30.21%
traffic sign        : 48.75%
vegetation          : 87.47%
terrain             : 50.39%
sky                 : 90.82%
person   

In [ ]:
train_miou

[20.63291775740628,
 27.0737167754515,
 29.561435231202697,
 33.907613993078144,
 36.79209028266278,
 39.35792562523391,
 41.35330127229657,
 43.27094127827265,
 44.77171953888862,
 47.22393651386267,
 50.11119294962304,
 52.03078326702902,
 54.28677040473266,
 58.16815881391402,
 59.60357576560984,
 61.46129634217924,
 63.543563148338386,
 65.84105499529812,
 67.17524801474222,
 68.8191620919247,
 71.05489872318579,
 71.62062424733226,
 72.30716628044343,
 73.11151646617866,
 74.35554695883079,
 74.57579843522386,
 75.17804555146576,
 75.64142425450504,
 76.37854149870319,
 76.45192597403074,
 77.02037369118048,
 77.38158559069085,
 77.60893371874671,
 77.84495316663751,
 78.30211429319523,
 78.57934878855056,
 78.75326079909274,
 79.01679606213958,
 79.27735379585317,
 79.29069460997165,
 79.62183578746333,
 79.76301405156075,
 79.89302340152895,
 80.05761696368583,
 80.07011283724952,
 80.26631403996592,
 80.37778345942895,
 80.4640991813969,
 80.52219847464231,
 80.59390524536818]

In [ ]:
[i for i,j in enumerate(train_miou) if j==max(train_miou)]

[49]

In [ ]:
val_miou

[24.062684416966764,
 29.053007990088897,
 30.100233825495586,
 34.743978193739814,
 35.91843137679799,
 37.899479853269966,
 37.09506858950151,
 36.885226783723056,
 41.869840660305215,
 43.43082500686602,
 41.79361145342313,
 40.08527258620049,
 46.005586757190756,
 47.39026867489633,
 47.2208490623228,
 45.94758923033333,
 49.70775091841716,
 49.401778712424765,
 50.53617783207272,
 52.765001478953636,
 50.350513057234494,
 52.202194000786776,
 51.861756029245356,
 52.722419921755005,
 52.94430068501561,
 53.5140990299771,
 52.866086296036876,
 53.45727323308612,
 53.16140551439453,
 53.58189228400255,
 53.80060894525971,
 53.90575388200594,
 54.19942716808078,
 53.38016718968046,
 52.83917303727691,
 53.96860490439087,
 53.680359331335396,
 53.656929278298115,
 53.46330183893443,
 53.91248550014384,
 53.83362138441494,
 53.724287411652526,
 53.43536840217493,
 53.621531952424014,
 53.39313342554236,
 53.58857299131595,
 53.42275206893359,
 53.71868748913323,
 53.364911677343315,
 5

In [ ]:
best_epoch

32

In [ ]:
print(test_fps_latency(BiSeNet_model, device))

{'mean_latency': 4.287259578704834, 'std_latency': 0.1466949712626179, 'min_latency': 4.127264022827148, 'max_latency': 4.808664321899414, 'fps': 233.24923103958554}


In [ ]:
print(count_parameters(BiSeNet_model))

{'total_params': 12581672, 'trainable_params': 12581672, 'params_M': 12.581672}


In [21]:
print(test_FLOPs_params(BiSeNet_model, device, 64, 128))

| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| model                                       | 12.582M                | 3.265G     |
|  saptial_path                               |  0.371M                |  0.641G    |
|   saptial_path.convblock1                   |   1.856K               |   33.554M  |
|    saptial_path.convblock1.conv1            |    1.728K              |    28.312M |
|    saptial_path.convblock1.bn               |    0.128K              |    5.243M  |
|   saptial_path.convblock2                   |   73.984K              |   0.305G   |
|    saptial_path.convblock2.conv1            |    73.728K             |    0.302G  |
|    saptial_path.convblock2.bn               |    0.256K              |    2.621M  |
|   saptial_path.convblock3                   |   0.295M               |   0.303G   |
|    saptial_path.convblock3.conv1            |    0.2